# Dicitonary Functionals

Provides functionals for diciontary related stuff. In most cases, named tuples are also supported.

I'm not too sure about the jargon here: I'm using _functional_ to mean a function that returns a function. Typically, a functional's parameters determine the parameter of the function that's returned.

In [1]:
"""Dictionary functionals contains functions that return dictionary manipulating functions.

Functionals take zero or more parameters that specify how to manipulate a dictionary. Their returned 
functions take one or more dictionaries and return a value.

Functionals
-----------
has_keys:
    Tests whether dictionary has the keys in a list.

filter_values:
    Tests whether values in a dictionary satisfy a predicate.

extract_keys:
    Return a dictionary including only certain keys.

drop_keys:
    Return a dictionary excluding certain keys.

map_values:
    Apply a mapping to specified keys, returning the transformed dictionary.

flatten_dictionary:
    Return a dictionary that inherits the key-value pairs of any dictionary-valued key.

"""

'Dictionary functionals contains functions that return dictionary manipulating functions.\n\nFunctionals take zero or more parameters that specify how to manipulate a dictionary. Their returned \nfunctions take one or more dictionaries and return a value.\n\nFunctionals\n-----------\nhas_keys:\n    Tests whether dictionary has the keys in a list.\n\nfilter_values:\n    Tests whether values in a dictionary satisfy a predicate.\n\nextract_keys:\n    Return a dictionary including only certain keys.\n\ndrop_keys:\n    Return a dictionary excluding certain keys.\n\nmap_values:\n    Apply a mapping to specified keys, returning the transformed dictionary.\n\nflatten_dictionary:\n    Return a dictionary that inherits the key-value pairs of any dictionary-valued key.\n\n'

In [10]:
from typing import List, Callable, Dict, Any
from functools import singledispatch, lru_cache
from collections import namedtuple

from functionals.dict_functionals import has_keys, extract_keys, drop_keys, map_values, filter_values, flatten_dict
from functions.utils import nt_builder

ImportError: attempted relative import with no known parent package

In [4]:
FunctionalMap = Dict[Any, Callable]

In [5]:
test_nt = namedtuple('test_nt', ['a', 'b'])
d = {'a': lambda x: x > 2}
assert not filter_values(d)({'a': 1})
assert filter_values(d)({'a': 3})
assert filter_values(d)(test_nt(3, 1))


In [6]:
list_of_dicts = [
    {'a': 1, 'b': 2},
    {'c': 2, 'b': 3},
    {'a': 2, 'b': 3, 'c': 0}
]

has_keys_test = list(filter(has_keys(['b'], every=True), list_of_dicts))
assert len(has_keys_test) == 3
assert all([('b' in dict_) for dict_ in has_keys_test])

In [23]:
nt_version = namedtuple('test', ['a', 'b'])(1, 0)
assert drop_keys(['a'])({'a': 1, 'b': 0}) == {'b': 0}
assert drop_keys(['a'])(nt_version) == namedtuple('dropped', ['b'])(0)
assert extract_keys(['a'])({'a': 1, 'b': 0}) == {'a': 1}
assert extract_keys(['a'])(nt_version) == namedtuple('extracted', ['a'])(1)
assert map_values({'a': lambda x: x + 1})({'a': 1, 'b': 0}) == {'a': 2, 'b': 0}
assert map_values({'a': lambda x: x + 1})(nt_version) == namedtuple('test', ['a', 'b'])(2, 0)

In [32]:
%timeit map_values({'a': lambda x: x**2})(test_nt(3, 1))
#isinstance(test_nt(3, 1)       , tuple)
        


22.6 µs ± 88.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%timeit map_values({'a': lambda x: x**2})({'a': 3, 'b': 1, 'c': 2})

In [ ]:
assert flatten_dict()({"a": 1, 'd': {"b":1, 'c': 1}}) == {'a': 1, 'd__b': 1, 'd__c': 1}

{'a': 1, 'd__b': 1, 'd__c': 1}

In [7]:

# UnboundLocalError in Python
var = 20 # a is global variable

def my_function():
    print(var)
    var = 'Hello' # This causes UnboundLocalError
    print(var)

my_function() # Calling function


UnboundLocalError: local variable 'var' referenced before assignment

In [ ]:
import math
def sequential_func(*functions):
    """Apply functions in order."""
    def _func(*args):
        for function in functions:
            try:
                args = function(*args)
            except TypeError:
                args = function(args)
        return args
    return _func


In [12]:
def explode_dict(dictionary, keys: list, acc=[], except_func=lambda x: not isinstance(x, list)):
    """Return an iterable of dictionaries from a dictionary with iterable values.

    Parameters
    ----------
    key
        The key(s) to explode.

    Returns
    -------
        A generator yielding the exploded dictionaries.

    Example
    -------
        list(explode_dict({a: [1, 2], b: [3, 4]}, ['a']))
        >>> [{a: 1, b: 3}, {a: 2, b: 3}, {a: 1, b: 4}, {a: 2, b: 4}]
    """

    def _yield_values(d, key):
        if not hasattr(d[key], '__iter__') or except_func(d[key]):
            yield d[key]
        else:
            yield from d[key]

    if not acc:
        return explode_dict({}, keys, [dictionary])
    if len(keys) == 1:
        return ({**d, keys[0]: v} for d in acc for v in _yield_values(d, keys[0]))
    acc = explode_dict({}, keys[:1], acc)
    return explode_dict({}, keys[1:], acc)


In [16]:
list(explode_dict(test_case, [1, 2]))

[{1: 0, 2: 2},
 {1: 0, 2: 0},
 {1: 1, 2: 2},
 {1: 1, 2: 0},
 {1: 2, 2: 2},
 {1: 2, 2: 0}]

In [18]:
test_case = {1: [0, 1, 2], 2: [2, 0]}
assert list(explode_dict(test_case, [1, 2])) == [
    {1: 0, 2: 2},
    {1: 0, 2: 0},
    {1: 1, 2: 2},
    {1: 1, 2: 0},
    {1: 2, 2: 2},
    {1: 2, 2: 0}
]
test_case2 = {1: 1, 2: [2, 3]}
assert list(explode_dict(test_case2, [1])) == [test_case2]